In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign
from IPython.display import display, HTML, Math

In [2]:
delta_b_negative = False
discrete = False
simp = True

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'l', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, l_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, l_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

in the contract, where Delta_b is negative:

-Delta_b * s_s <= Delta_s * s_b

implying for positive Delta_b:

Delta_b * s_s <= Delta_s * s_b

which maximizes value for the trader if:

In [4]:
if delta_b_negative:
  a0_eq_Delta = Eq(-Delta_b * s_s, Delta_s * s_b)
else:
  a0_eq_Delta = Eq(Delta_b * s_s, Delta_s * s_b)
a0_eq_Delta

Eq(Delta_b*s_s, Delta_s*s_b)

In [5]:
if delta_b_negative:
  a0_eq_Value = Eq(l_b * s_s + l_s * s_b, (l_b + Delta_b) * s_s + (l_s + Delta_s) * s_b)
else:
  a0_eq_Value = Eq(l_b * s_s + l_s * s_b, (l_b - Delta_b) * s_s + (l_s + Delta_s) * s_b)
a0_eq_Value

Eq(l_b*s_s + l_s*s_b, s_b*(Delta_s + l_s) + s_s*(-Delta_b + l_b))

In [6]:
assert(solveset(a0_eq_Value, Delta_b).simplify() == solveset(a0_eq_Delta, Delta_b))
assert(solveset(a0_eq_Value, Delta_s).simplify() == solveset(a0_eq_Delta, Delta_s))
a0_eq = a0_eq_Delta

In [65]:
# this seems to be derived from spotByDelta
def deltaBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = (s - l * w) / w # at this point, this value is negative for buying and positive for selling
  # rounding towards zero in all cases, as we are interested in the maximum delta-capacity of a given spot price
  if asset == 'b':
    if delta_b_negative:
      if discrete:
        f = ceiling(f)
    else:
      if discrete:
        f = floor(-f)
      else:
        f = -f

  else:
    if discrete:
      f = floor(f)

  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(Delta_b, deltaBySpot['b']))
display(Eq(Delta_s, deltaBySpot['s']))

Eq(Delta_b, l_b - s_b/w_b)

Eq(Delta_s, -l_s + s_s/w_s)

In [9]:
solveset(Eq(Delta_b, deltaBySpot['b']), s_b)

{-w_b*(Delta_b - l_b)}

In [10]:
solveset(Eq(Delta_s, deltaBySpot['s']), s_s)

{w_s*(Delta_s + l_s)}

### logic behind spotByDelta

- we need to pick a spot price that is worse than the amm price = w * l
- for uninverted prices, worse means larger for buying and lower for selling
- since prices are inverted, worse means smaller for buying and larger for selling
- this must hold true after the trade as well, so it must also be larger than w * (l + Delta) for selling and smaller than w * (l - Delta) for buying 
- (assuming Delta_b is represented positively, otherwise it's larger (selling) resp. smaller (buying) than w * (l + Delta) for both)

In [11]:
def spotByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  if (not delta_b_negative) and asset == 'b':
    f = w * (l - Delta)
  else:
    f = w * (l + Delta)
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(s_b, spotByDelta['b']))
display(Eq(s_s, spotByDelta['s']))

Eq(s_b, w_b*(-Delta_b + l_b))

Eq(s_s, w_s*(Delta_s + l_s))

In [12]:
assert(Eq(spotByDelta['b'].subs({Delta_b: deltaBySpot['b']}), s_b))
assert(Eq(spotByDelta['s'].subs({Delta_s: deltaBySpot['s']}), s_s))
if discrete:
  display(deltaBySpot['b'].subs({s_b: spotByDelta['b']}))
  display(deltaBySpot['s'].subs({s_s: spotByDelta['s']}))
else:
  assert(Eq(deltaBySpot['b'].subs({s_b: spotByDelta['b']}), Delta_b))
  assert(Eq(deltaBySpot['s'].subs({s_s: spotByDelta['s']}), Delta_s))

In [13]:
a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}).simplify()

Eq(Delta_b*w_s*(Delta_s + l_s), Delta_s*w_b*(-Delta_b + l_b))

In [14]:
def deltaByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  # if discrete:
  #   if asset == 'b' and delta_b_negative:
  #     domain = S.Integers
  #   else:
  #     domain = S.Naturals
  # else:
  #   domain = S.Reals
  # above takes more than 11 minutes, skipping for now
  f = solveset(a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}), Delta)#, domain=domain)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if discrete:
    if asset == 'b':
      f = floor(f)
    else:
      f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaByDelta = {asset: deltaByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(Delta_b, deltaByDelta['b']))
display(Eq(Delta_s, deltaByDelta['s']))

Eq(Delta_b, Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s))

Eq(Delta_s, -Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b))

### note:

 - the above is undefined/negative (which reduces to undefined considering the meaning) for some values.
 - when we represent Delta_b negatively, this issue appears to vanish, but actually doesn't when pondering the sign of Delta_b

In [15]:
denominator_Delta_b = deltaByDelta['b'].args[-1].args[0]
display(denominator_Delta_b)
ndef_Delta_s = solveset(denominator_Delta_b, Delta_s)
assert(len(ndef_Delta_s) == 1)
ndef_Delta_s = list(ndef_Delta_s)[0]
ndef_Delta_s

Delta_s*w_b + Delta_s*w_s + l_s*w_s

-l_s*w_s/(w_b + w_s)

In [16]:
denominator_Delta_s = deltaByDelta['s'].args[-1].args[0]
display(denominator_Delta_s)
ndef_Delta_b = solveset(denominator_Delta_s, Delta_b)
assert(len(ndef_Delta_b) == 1)
ndef_Delta_b = list(ndef_Delta_b)[0]
ndef_Delta_b

Delta_b*w_b + Delta_b*w_s - l_b*w_b

l_b*w_b/(w_b + w_s)

In [31]:
display(solveset(a0_eq, Delta_s))
display(solveset(a0_eq, Delta_b))

{Delta_b*s_s/s_b}

{Delta_s*s_b/s_s}

given that we can lower s_b arbitrarily (up to a lower bound of 1) and increase s_s arbitrarily, we should always be able to get a positive Delta_s


given that we can get some negative Delta_s, does this imply an exploit?

- in the onchain-code, the swap prices can never be negative, as they are computed there from exponents
- likewise, if Delta_b is negative (as it should be there), -Delta_b * s_s is positive
- -> negative Delta_s would violate -Delta_b * s_s <= Delta_s * s_b (for negative Delta_b)

-> no exploit onchain (except potentially negating both deltas, which we examine further below)

### how do we adjust the equations for the spot prices accordingly, such that the user still get's the best effective price?
And after we found an equation, how does it reconcile with the one we got before?


In [73]:
lowest_spot = (s_s / s_b).subs({s_s: spotByDelta['s'], s_b: spotByDelta['b']})#.simplify()
lowest_spot

w_s*(Delta_s + l_s)/(w_b*(-Delta_b + l_b))

lowest_spot: 
- best (lowest) uninverted spot price the user can get (with highest inverted buying price and lowest inverted selling price)
- evidently always positive

In [77]:
Ge(Delta_s, (Delta_b * lowest_spot))#.simplify()

Delta_s >= Delta_b*w_s*(Delta_s + l_s)/(w_b*(-Delta_b + l_b))

above:
- as Delta_b is positive (in the respective representation) and spot is positive, Delta_s should always be positive
- this however is just a rearrangement of what we already have, resulting in the same paradox

In [91]:
solveset(a0_eq.subs({s_s: deltaBySpot['s'], Delta_b: ndef_Delta_b}).simplify(), s_b)

{-l_b*w_b*(l_s*w_s - s_s)/(Delta_s*w_s*(w_b + w_s))}

In [94]:
solveset(a0_eq.subs({s_b: deltaBySpot['b'], Delta_b: ndef_Delta_b}).simplify(), s_s)

{Delta_s*(w_b + w_s)*(l_b*w_b - s_b)/(l_b*w_b**2)}

In [95]:
solveset(a0_eq.subs({s_s: deltaBySpot['s'], Delta_b: ndef_Delta_b}).simplify(), s_s)

{Delta_s*s_b*w_s*(w_b + w_s)/(l_b*w_b) + l_s*w_s}

In [96]:
solveset(a0_eq.subs({s_b: deltaBySpot['b'], Delta_b: ndef_Delta_b}).simplify(), s_b)

{l_b*w_b - l_b*s_s*w_b**2/(Delta_s*(w_b + w_s))}

TODO continue here

(below is just some check that fits nowhere really)

In [18]:
solveset(Eq(Delta_s, deltaByDelta['s']), Delta_b)

Complement({Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s)}, {l_b*w_b/(w_b + w_s)})

In [19]:
solveset(Eq(Delta_b, deltaByDelta['b']), Delta_s)

Complement({-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)}, {-l_s*w_s/(w_b + w_s)})

### TODO what happens if an attacker simply removes the "selling" asset and adds the "buying" one instead of the other way around?

In [20]:
def spotBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = solveset(a0_eq.subs({Delta_b: deltaBySpot['b'], Delta_s: deltaBySpot['s']}), s)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if simp:
    f = simplify(f)
  return f

spotBySpot = {asset: spotBySpot_(asset, *all_symbols[asset]) for asset in assets}
spotBySpot['b']

l_b*s_s*w_b*w_s/(-l_s*w_b*w_s + s_s*w_b + s_s*w_s)

In [21]:
spotBySpot['s']

l_s*s_b*w_b*w_s/(-l_b*w_b*w_s + s_b*w_b + s_b*w_s)

all four solutions are the same irrespective of delta_b_negative (continuous case, and not accounting for flipped indices, of course)


In [22]:
denominator_s_s = spotBySpot['s'].args[-1].args[0]
display(denominator_s_s)
ndef_s_b = solveset(denominator_s_s, s_b)
assert(len(ndef_s_b) == 1)
ndef_s_b = ndef_s_b.args[0]
ndef_s_b

-l_b*w_b*w_s + s_b*w_b + s_b*w_s

l_b*w_b*w_s/(w_b + w_s)

In [23]:
denominator_s_b = spotBySpot['b'].args[-1].args[0]
display(denominator_s_b)
ndef_s_s = solveset(denominator_s_b, s_s)
assert(len(ndef_s_s) == 1)
ndef_s_s = ndef_s_s.args[0]
ndef_s_s

-l_s*w_b*w_s + s_s*w_b + s_s*w_s

l_s*w_b*w_s/(w_b + w_s)

In [24]:
display(ndef_Delta_b)
display(ndef_Delta_s)

l_b*w_b/(w_b + w_s)

-l_s*w_s/(w_b + w_s)

In [25]:
display(ndef_Delta_b * -w_s)
display(ndef_Delta_s * -w_b)

-l_b*w_b*w_s/(w_b + w_s)

l_s*w_b*w_s/(w_b + w_s)

- the implied limits on spot prices look similar to those on Delta_b
- this similarity is encapsulated by negation and multiplication by the other asset's weight, which is odd, but reminds us a bit of the amm price calculation

what happens if we use those invalid spot prices in our other equations? Do they break or not?

In [26]:
ndef_s_s = solveset(spotBySpot['b'].args[-1].args[0], s_s)
assert(len(ndef_s_s) == 1)
ndef_s_s = ndef_s_s.args[0]
ndef_s_s

l_s*w_b*w_s/(w_b + w_s)

In [27]:
a0_ndef_spots = a0_eq.subs({s_b: ndef_s_b, s_s: ndef_s_s})
a0_ndef_spots

Eq(Delta_b*l_s*w_b*w_s/(w_b + w_s), Delta_s*l_b*w_b*w_s/(w_b + w_s))

In [28]:
solveset(a0_ndef_spots, Delta_b)

{Delta_s*l_b/l_s}

In [29]:
solveset(a0_eq, Delta_s)

{Delta_b*s_s/s_b}

In [30]:
solveset(a0_ndef_spots, Delta_s)

{Delta_b*l_s/l_b}